In [ ]:
from ExperimentalDataReader import DataReader
from matplotlib import pyplot as plt
from os import listdir
import pandas as pd
import regex as re

%matplotlib inline

In [ ]:
def get_meta_data(datasets_folder='datasets/raw_data/', integral_time=3.0, time_between_meas=3.5):
    
    def get_file_names(datasets_folder=datasets_folder):
        files=pd.DataFrame({'file_names': listdir(datasets_folder)})

        data_files = files.loc[~files.file_names.str.startswith('Температура')]
        temperature_files = files.loc[files.file_names.str.startswith('Температура')].reset_index()

        files = pd.DataFrame({'data_files': data_files.file_names,
                              'temperature_files': temperature_files.file_names
                             })

        return files
    
    def get_params(files, integral_time, time_between_meas):
        files['patterns'] = files.data_files.str.split('_')
        
        get_spc_name = lambda x: ' '.join(x[:2])
        files['specimen_name'] = files.patterns.apply(get_spc_name)
        
        get_bs = lambda x: int(x[3].strip('пФ'))
        files['bs'] = files.patterns.apply(get_bs)
        
        get_u1 = lambda x: float(re.findall(string=x[5], pattern=r'[+-]?\d+\.?\d*')[0])
        files['u1'] = files.patterns.apply(get_u1)
        
        get_ur = lambda x: float(re.findall(string=x[5], pattern=r'[+-]?\d+\.?\d*')[-1])
        files['ur'] = files.patterns.apply(get_ur)
        
        get_ls = lambda x: int(x[6].strip('мВ'))
        files['ls'] = files.patterns.apply(get_ls)
        
        get_f_pulse = lambda x: float(x[7].strip('мкс'))
        files['f_pulse'] = files.patterns.apply(get_f_pulse)
        
        files['time_between_meas'] = time_between_meas
        files['integral_time'] = integral_time
        
        return files.drop('patterns', axis='columns')
    
    files = get_file_names()
    return get_params(files, integral_time, time_between_meas)

In [ ]:
def prepare_data(meta_data, folder='datasets/raw_data'):
    data_reader = DataReader()
    
    folder = folder
    d_file_name = folder + '/' + meta_data.data_files
    t_file_name = folder + '/' + meta_data.temperature_files

    data_reader.read_from_d_t(d_file_name=d_file_name, t_file_name=t_file_name)

    data_reader.set_bs(meta_data.bs)

    data_reader.set_ls(meta_data.ls)

    data_reader.set_f_pulse(meta_data.f_pulse)

    data_reader.compute_dlts_pf()

    data_reader.set_specimen_name(meta_data.specimen_name)

    data_reader.set_u1(meta_data.u1)

    data_reader.set_ur(meta_data.ur)

    data_reader.set_time_between_meas(meta_data.time_between_meas)

    data_reader.set_integral_time(meta_data.integral_time)
    
    return data_reader

In [ ]:
meta_data = get_meta_data()

In [ ]:
data_readers = meta_data.apply(prepare_data, axis='columns')

folder = 'datasets'

for i, data_reader in enumerate(data_readers):
    data_reader.to_csv(folder + '/' + meta_data.data_files[i] + '.csv')
    
    data_reader.get_plot()
    plt.savefig(folder + '/' + meta_data.data_files[i] + '.jpg', bbox_inches='tight')
    plt.savefig(folder + '/' + meta_data.data_files[i] + '.pdf', bbox_inches='tight')
    
    plt.close('all')
    print(f'№{i}\t{meta_data.data_files[i]}\t- OK')